In [1]:
import datetime, time, math, os, json, shutil, asyncio, networkx as nx
from utils.gitutils import create_pull_request, clone_repo, create_branch, wipe_repo, prepare_repo
from utils.agent import Agent, GenerationConfig, Interaction, Team
from utils.stringutils import arr_from_sep_string, extract_markdown_blocks, markdown_to_dict, closest_substr
from utils.filetreeutils import FileTree, write_file_tree
from utils.listutils import flatten
from utils.frameworkutils import DartAnalyzer
from dotenv import load_dotenv
from utils.graphutils import loose_level_order, collapsed_level_order
from utils.frameworkutils import Framework

In [2]:
load_dotenv()  # Load environment variables from .env

GITHUB_TOKEN = os.getenv('GITHUB_TOKEN')
API_KEY = os.getenv('API_KEY')

In [3]:
repo_url = 'https://github.com/6165-MSET-CuttleFish/TeamTrack'
source = Framework.FLUTTER
feature = f"""## Name
Organizations
## Description
Organizations are a way to group users together. They can be used to automatically share events with the members of the organization whenever they are uploaded to the cloud. \
Organizations consist of a name, team number, and a backend-generated unique id. They can also have a profile picture, description, and location. \
Members can also opt to be publicly listed as a member of the organization or not. \
"""

In [23]:
# Clone the repo and make a new branch
repo = clone_repo(repo_url)
base_branch = repo.active_branch.name
created_branch = f"modification-{base_branch}"
local_repo_path = str(repo.working_dir)
working_dir_path = f'{local_repo_path}\\{source.get_working_dir()}'

In [5]:
source_swe_summarizer = Agent(
    model_name="gemini-1.5-flash-001",
    api_key=API_KEY,
    name="source_swe_analyzer",
    generation_config=GenerationConfig(temperature=0.7),
    system_prompt=f'''You are a {source} software engineer. Your job is to summarize the functionality of provided code files. Structure your response as follows:

# filename.ext:

$7 sentence summary of the functionality/contents of the file$
'''
)

pm = Agent(
    model_name="gemini-1.5-pro-001",
    api_key=API_KEY,
    name="pm",
    generation_config=GenerationConfig(temperature=0.3),
    system_prompt=f"""You are a high-level technical project manager tasked with the project of adding functionality to a {source} repository."""
)

source_swe_coder = Agent(
    model_name="gemini-1.5-flash-001",
    api_key=API_KEY,
    name="source_swe_coder",
    generation_config=GenerationConfig(temperature=0.5),
    system_prompt=f"You are a {source} software engineer. Your job is to write code to implement a specific feature. Respond with code and nothing else. No explanations needed."
)

team = Team(source_swe_summarizer, pm, source_swe_coder)

In [24]:
analyzer = source.get_analyzer(working_dir_path)

source_dependency_graph = analyzer.buildDependencyGraph()

In [7]:
# loose level order
eval_order = loose_level_order(source_dependency_graph)[::-1]
for level in eval_order:
    print(level)
print(f"Number of levels: {len(eval_order)}")

['models\\StatConfig.dart', 'models\\AppModel.dart']
['models\\GameModel.dart']
['models\\ScoreModel.dart']
['functions\\Extensions.dart', 'components\\misc\\PlatformGraphics.dart']
['components\\statistics\\BarGraph.dart', 'components\\users\\PFP.dart']
['providers\\Auth.dart', 'components\\users\\UsersRow.dart', 'functions\\Statistics.dart', 'components\\scores\\Incrementor.dart', 'components\\scores\\ScoreSummary.dart']
['views\\home\\match\\MatchView.dart']
['components\\statistics\\PercentChange.dart', 'functions\\Functions.dart']
['components\\scores\\ScoreRangeSummary.dart', 'models\\Change.dart', 'components\\scores\\ScoreTimeline.dart']
['views\\home\\change\\ChangeConfig.dart', 'views\\home\\match\\MatchRow.dart', 'views\\home\\match\\MatchConfig.dart', 'components\\scores\\ScoringElementStats.dart', 'components\\misc\\CardView.dart', 'components\\statistics\\CheckList.dart', 'views\\home\\change\\ChangeRow.dart', 'components\\misc\\EmptyList.dart', 'views\\home\\match\\Examp

In [8]:
(source_file_tree := FileTree.from_dir(working_dir_path))

├── api\
│   ├── README.MD
├── components\
│   ├── misc\
│   │   ├── CardView.dart
│   │   ├── Collapsible.dart
│   │   ├── EmptyList.dart
│   │   ├── InfoPills.dart
│   │   ├── PlatformGraphics.dart
│   ├── scores\
│   │   ├── Incrementor.dart
│   │   ├── ScoreCard.dart
│   │   ├── ScoreRangeSummary.dart
│   │   ├── ScoreSummary.dart
│   │   ├── ScoreTimeline.dart
│   │   ├── ScoringElementStats.dart
│   ├── statistics\
│   │   ├── BarGraph.dart
│   │   ├── CheckList.dart
│   │   ├── PercentChange.dart
│   ├── users\
│   │   ├── PFP.dart
│   │   ├── UsersRow.dart
│   ├── AutonomousDrawingTool.dart
├── functions\
│   ├── APIMethods.dart
│   ├── Extensions.dart
│   ├── Functions.dart
│   ├── GPTModel.dart
│   ├── ResponseModel.dart
│   ├── Statistics.dart
├── models\
│   ├── AppModel.dart
│   ├── Change.dart
│   ├── GameModel.dart
│   ├── GPTModel.dart
│   ├── ScoreModel.dart
│   ├── StatConfig.dart
├── providers\
│   ├── Auth.dart
│   ├── PushNotifications.dart
│   ├── Theme.dart
│   ├

In [9]:
# Summarize the original repo
from tqdm import tqdm

async def summarize_group(group):
    async def summarize(node):
        name = source_file_tree.nodes[node]['name']
        content = source_file_tree.nodes[node]['content']
        message = f"{name}\n```\n{content}\n```"
        return await team.async_chat_with_agent(
            agent_name='source_swe_analyzer',
            message=message,
            context_keys=[f"summary_{neighbor}" for neighbor in source_dependency_graph[node]],
            save_keys=[f"summary_{node}", "all"],
            prompt_title=f"Summary of {node}"
            )
    tasks = [summarize(node) for node in group]
    responses = await asyncio.gather(*tasks)
    for i, response in enumerate(responses):
        source_file_tree.nodes[group[i]]["summary"] = response

for level in tqdm(eval_order):
    await summarize_group(level)
    time.sleep(0.5)

100%|██████████| 19/19 [01:16<00:00,  4.04s/it]


In [10]:
prompt = f"""This is the file tree for the original {source} repo:
```
{source.get_working_dir()}\\
{source_file_tree}
```
Given the prior context, summarize the functionality of the entire repository succinctly.
"""
print(team.chat_with_agent('source_swe_analyzer', prompt, context_keys=["all"], save_keys=["all"], prompt_title=f"""Repository Summary.
File Tree provided:
```
{source.get_working_dir()}\\
{source_file_tree}
```
"""))

This Flutter repository implements a comprehensive application for managing and analyzing data related to First Tech Challenge (FTC) robotics competitions. The app allows users to:

* **Manage Events:** Create, share, edit, and delete events.
* **Track Matches:** Add, view, and edit match data, including scores, penalties, and active users.
* **Analyze Team Performance:** View team statistics, sort teams by various criteria, and analyze their performance using statistical tools and visualizations.
* **Draw Autonomous Paths:** Draw autonomous paths for specific teams on a simulated field image.
* **Simulate Alliances:** Explore various alliance combinations and receive recommendations using a GPT model.
* **Share Events:** Invite other users to join events and manage their permissions.
* **Manage Inbox:** View and manage event invitations, block senders, and accept invitations.
* **Utilize Templates:** Access event templates from The Orange Alliance API to quickly create new events.
* *

In [11]:
curr_string = '\n'.join([message['parts'][0] for interaction in team.context_threads['all'] for message in interaction.to_dict()])
(curr_tokens := pm.model.count_tokens(curr_string).total_tokens)

19787

In [12]:
would_be_string = '\n'.join([source_file_tree.nodes[node]['content'] for node in source_dependency_graph.nodes])
(would_be_tokens := pm.model.count_tokens(would_be_string).total_tokens)

122951

In [13]:
(percent_tokens_reduction := 1 - (curr_tokens / would_be_tokens))

0.8390659693699116

In [14]:
response_format = f"""
You may choose ACTIONS from the following list:
- | WRITE | $FILE_PATH$ | $PROMPT$ (PROMPT should inform how the new FILE_PATH should be written)
- | EDIT | $FILE_PATH$ | $PROMPT$ (Edit the provided file to fulfill the prompt)
- | DELETE | $FILE_PATH$ (Delete the provided file)
----------
Here is a sample response:
```
| WRITE | {source.get_working_dir()}\\path\\to\\newfile.ext | Implement ... with the following properties:
    - property1
    - property2
| EDIT | {source.get_working_dir()}\\path\\to\\file.ext | Fix the bug ... by doing:
    - step1
    - step2
| DELETE | {source.get_working_dir()}\\path\\to\\some\\file.ext | Reasons for deletion:
    - reason1
    - reason2
```
"""

# Create new file tree
prompt = f'''
You are to build the following feature:
{feature}
------------------
{response_format}
------------------
You need to follow the response template structure exactly (MY JOB DEPENDS ON IT)
'''
print(response := team.chat_with_agent("pm", prompt, context_keys=["all"], save_keys=["all"], prompt_title="Feature Creation Steps"))

```
| WRITE | lib/models/Organization.dart | Implement the `Organization` class with the following properties:
    - `id`: String (backend-generated unique ID)
    - `name`: String
    - `teamNumber`: String
    - `pictureUrl`: String? (nullable, for profile picture)
    - `description`: String? (nullable)
    - `location`: String? (nullable)
| WRITE | lib/providers/OrganizationProvider.dart | Implement the `OrganizationProvider` class that handles:
    - Creating new organizations.
    - Fetching organization details.
    - Updating organization information.
    - Listing organizations for a user.
    - Managing organization members.
| WRITE | lib/views/organization/OrganizationScreen.dart | Implement the `OrganizationScreen` widget that displays:
    - A list of organizations the user is a member of.
    - Options to create a new organization or join an existing one.
| WRITE | lib/views/organization/OrganizationDetails.dart | Implement the `OrganizationDetails` widget that displays:


In [15]:
md = extract_markdown_blocks(response)[0]
blocks = md.split("|")
actions = []
i = 0
while i < len(blocks):
    block = blocks[i].strip()
    if block != "WRITE" and block != "EDIT" and block != "DELETE":
        i += 1
        continue
    action, file, prompt = blocks[i:i + 3]
    action = action.strip()
    file = file.strip()[len(source.get_working_dir()) + 1:]
    prompt = prompt.strip()
    actions.append((action, file, prompt))
    i += 3
for action in actions:
    print(action)

('WRITE', 'models/Organization.dart', 'Implement the `Organization` class with the following properties:\n    - `id`: String (backend-generated unique ID)\n    - `name`: String\n    - `teamNumber`: String\n    - `pictureUrl`: String? (nullable, for profile picture)\n    - `description`: String? (nullable)\n    - `location`: String? (nullable)')
('WRITE', 'providers/OrganizationProvider.dart', 'Implement the `OrganizationProvider` class that handles:\n    - Creating new organizations.\n    - Fetching organization details.\n    - Updating organization information.\n    - Listing organizations for a user.\n    - Managing organization members.')
('WRITE', 'views/organization/OrganizationScreen.dart', 'Implement the `OrganizationScreen` widget that displays:\n    - A list of organizations the user is a member of.\n    - Options to create a new organization or join an existing one.')
('WRITE', 'views/organization/OrganizationDetails.dart', 'Implement the `OrganizationDetails` widget that dis

## Context Caching

In [16]:
source_swe_coder.model.count_tokens('\n'.join(message['parts'][0] for interaction in team.context_threads['all'] for message in interaction.to_dict())).total_tokens

20169

In [17]:
# Cache the context threads
factor = 32768 / source_swe_coder.model.count_tokens('\n'.join(message['parts'][0] for interaction in team.context_threads['all'] for message in interaction.to_dict())).total_tokens
# eg. if factor is 3.2, round to 4
factor = math.ceil(factor)
thread = team.context_threads['all'].copy()
thread *= factor
source_swe_coder.model.count_tokens('\n'.join(message['parts'][0] for interaction in thread for message in interaction.to_dict())).total_tokens

40307

In [18]:
# source_swe_coder.cache_context(thread, ttl=datetime.timedelta(minutes=8))

In [21]:
def write(file_path, prompt):
    response = source_swe_coder.chat(f"Instructions for {file_path}:\n{prompt}")
    md = extract_markdown_blocks(response)[0]
    path = f"{working_dir_path}\\{file_path}"
    if not os.path.exists(os.path.dirname(path)):
        os.makedirs(os.path.dirname(path))
    with open(path, 'w') as f:
        f.write(md)

def edit(file_path, prompt):
    with open(f"{working_dir_path}\\{file_path}", 'r') as f:
        content = f.read()
    response = source_swe_coder.chat(f"""{prompt}
Structure your response as follows:
## Short Old Code Snippet 1
```
$short old code snippet that needs to changed$
```
## New Code Snippet 1
```
$the new code snippet needed to fulfill the prompt$
```
## Short Old Code Snippet 2
```
$...$
```
## New Code Snippet 2
```
$...$
...
...
## Short Old Code Snippet N
```
$...$
```
## New Code Snippet N
```
$...$
```.  Respond with as many or as few snippets as you need. DO NOT RESPOND WITH A DIFF FILE OR IN A DIFFERENT FORMAT, MY JOB DEPENDS ON IT""", custom_context=[Interaction(f"Contents of {file_path}", content)])
    blocks = extract_markdown_blocks(response)
    # even blocks are old code, odd blocks are new code
    old_codes = [block for i, block in enumerate(blocks) if i % 2 == 0]
    new_codes = [block for i, block in enumerate(blocks) if i % 2 == 1]
    with open(f'{working_dir_path}\\{file_path}', 'w') as f:
        for i in range(len(old_codes)):
            old_code = old_codes[i]
            closest_match, distance = closest_substr(content, old_code)
            new_code = new_codes[i]
            print(f"Old Code: {old_code}")
            print(f"Closest match: {closest_match}")
            f.write(content.replace(closest_match, new_code))

def delete(file_path):
    os.remove(f"{working_dir_path}\\{file_path}")

In [22]:
from utils.errorutils import repeat_until_finish
from tqdm import tqdm

for i in tqdm(range(0, len(actions))):
    action, file, prompt = actions[i]
    if action == 'WRITE':
        repeat_until_finish(lambda: write(file, prompt), max_retries=2)
    elif action == 'EDIT':
        repeat_until_finish(lambda: edit(file, prompt), max_retries=2)
    elif action == 'DELETE':
        delete(file)

 71%|███████▏  | 5/7 [00:18<00:08,  4.12s/it]

Old Code: PopupMenuButton<String>(
                        onSelected: (String item) {
                          if (item == 'Share with Organizations') {
                            Navigator.push(
                              context,
                              MaterialPageRoute(
                                builder: (context) =>
                                    ShareWithOrganizationsScreen(
                                      event: widget.event,
                                    ),
                              ),
                            );
                          } else {
                            Share.share(
                                '${widget.event.title}\n${widget.event.description}\n${widget.event.location}\n${widget.event.date}\n${widget.event.time}');
                          }
                        },
                        itemBuilder: (BuildContext context) {
                          return [
                            PopupMenuItem(
   

 86%|████████▌ | 6/7 [00:19<00:03,  3.05s/it]

Old Code: class DataModel {
  // ... other fields
}
Closest match: class DataModel {
  // ... other fields
  Organization? selectedOrganization;


100%|██████████| 7/7 [00:23<00:00,  3.41s/it]

Old Code: body: Center(
        child: _selectedIndex == 0
            ? Text('Home Screen')
            : _selectedIndex == 1
                ? OrganizationScreen()
                : ProfileScreen(),
      ),
Closest match:       body: Center(
        child: _selectedIndex == 0
            ? Text('Home Screen')
            : _selectedIndex == 1
                ? OrganizationScreen()
                : ProfileScreen(),
      ),
Old Code: items: const [
    BottomNavigationBarItem(
      icon: Icon(Icons.home),
      label: 'Home',
    ),
    BottomNavigationBarItem(
      icon: Icon(Icons.person),
      label: 'Profile',
    ),
  ],
Closest match:         items: const [
          BottomNavigationBarItem(
            icon: Icon(Icons.home),
            label: 'Home',
          ),
          BottomNavigationBarItem(
            icon: Icon(Icons.business),
            label: 'Organizations',
          ),
          BottomNavigationBarItem(
Old Code: case 0:
        Navigator.pushReplacementN